In [ ]:
from molecules import PPS, PolyEthylene
from system import System
from simulation import Simulation
from forcefields import GAFF, OPLS_AA, OPLS_AA_PPS


import warnings
warnings.filterwarnings("ignore")

# Using the recipes in molecules.py:

molecules.py contains classes for different monomers.  
These classes can be thought of as recipes or templates
to quickly build up polymer chains of any length

In [ ]:
# Use the PPS template to create a 5mer PPS molecule
pps_chain = PPS(length=5)
pps_chain.visualize().show()

In [ ]:
# We can do the same with a Poly-ethylene chain
pe_chain = PolyEthylene(length=8)
pe_chain.visualize()

# Building up a system of polymers:
Pass in an instance of a polymer template, and a list of number of molecules and a list of molecule lengths

In [ ]:
sys = System(molecule=PolyEthylene, n_mols=[10], chain_lengths=[10], density=1.3, united_atom=True)

In [ ]:
# Call one of the configuration building functions; right now, we only have pack()
sys.pack()

In [ ]:
# Apply a foyer forcefield
sys.apply_forcefield(forcefield=GAFF())

In [ ]:
# We now have 2 different versions of our system. An mBuild Compound and a Parmed Structure
print(sys.system)
print()
print(sys.typed_system)

# Starting up a simulation of our polymer system
- Use the Simulation class to quickly get a simulation going.  
- Pass in a `System.typed_system` which has all the forcefield information
- Most of the simulation properties of interest are easily accessble, and changeable via methods in the Simulation class
- The integrator and integrator method is determined by which run funciton you call
```
run_NVT
run_NPT
run_NVE
run_langevin
```

In [ ]:
sim = Simulation(system=sys.typed_system)

In [ ]:
print(sim.nlist)
print()
print(sim.forcefield)
print()
print(sim.dt)

In [ ]:
# The integrator and integrator method are not set up until a run function is called for the first time:
sim.run_NVT(n_steps=0, kT=1, tau_kt=0.1)

In [ ]:
print(sim.integrator)

In [ ]:
print(sim.method)

In [ ]:
# Change simulation parameters like dt
print(sim.integrator.dt)
sim.dt = 0.0001
print(sim.integrator.dt)

In [ ]:
# Calling a run function again will update the method if needed
print(sim.method)
sim.run_NPT(n_steps=0, pressure=0.001, kT=1.0, tau_kt=0.01, tau_pressure=0.1)
print(sim.method)

# Combining the run functions to get an equilibrated system
- First, call the `run_shrink` function to get the box down to the desired density
- Next, run in the NPT ensemble to get an equilibrated volume
- Finally, run at NVT to get an equilibrated morphology

In [ ]:
# Build up a polyethylene system of 10 10mers, apply an OPLS forcefield
sys = System(molecule=PolyEthylene, n_mols=[10], chain_lengths=[10], density=1.3)
sys.pack()
sys.apply_forcefield(forcefield=GAFF())

In [ ]:
# We can use the target_box attribute to help with the shrink run
# Remember we'll have to convert from nm to angstroms
# Remember to account for the reference distance if using auto scaling
sys.target_box

In [ ]:
sim = Simulation(system=sys.typed_system, auto_scale=True)

In [ ]:
# This is in angstroms:
sim.ref_distance

In [ ]:
sim.run_shrink(
    kT=5.0,
    final_box_lengths=sys.target_box * 10 / sim.ref_distance,
    n_steps=1e4,
    tau_kt = 0.01,
    period=100,
)